In [1]:
# Importando librerías de interés
import pandas as pd
import numpy as np

from plotnine import ggplot, aes, geom_histogram, facet_grid, facet_wrap, geom_bar, geom_point
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.plotting import scatter_matrix

from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn import svm
from sklearn.tree import export_graphviz
from IPython.display import Image  
import pydotplus

In [2]:
df_defunciones = pd.read_csv('defunciones-filtrado.csv')
df_defunciones.rename(columns={'id_loc': 'localidad',
                   'anio_ocur': 'anio',
                   'area_ur': 'urbanizacion',
                   },
          inplace=True)
df_defunciones = df_defunciones[['localidad', 'anio', 'cond_resp']]

In [3]:
df_salud = pd.read_csv('df_salud_filtrada.csv')
df_salud = df_salud[['localidad', 'anio', 'actividad_vigorosa_promedio', 'actividad_moderada_promedio', 'actividad_caminar_promedio']]

In [4]:
df = df_defunciones.merge(df_salud, how='left', left_on=['localidad', 'anio'], right_on=['localidad', 'anio'])
df

,localidad,anio,cond_resp,actividad_vigorosa_promedio,actividad_moderada_promedio,actividad_caminar_promedio
0,10_10_1,2012,2,NaN,NaN,NaN
1,10_10_1,2016,1,NaN,NaN,NaN
2,10_10_29,2012,0,NaN,NaN,NaN
3,10_10_29,2016,1,NaN,NaN,NaN
4,10_10_33,2012,0,NaN,NaN,NaN
...,...,...,...,...,...,...
48773,9_9_74,2016,1,NaN,NaN,NaN
48774,9_9_7777,2012,1,NaN,NaN,NaN
48775,9_9_7777,2016,0,NaN,NaN,NaN
48776,9_9_9999,2012,1,NaN,NaN,NaN


In [5]:
# Crea objeto para imputar con mediana
imputer = SimpleImputer(strategy="median")

In [6]:
# Ajusta conjunto de datos al objeto para imputar 
imputer.fit(df.drop(['localidad', 'anio', 'cond_resp'], axis=1))

SimpleImputer(strategy='median')

In [7]:
imputer.statistics_

array([152.80247798, 364.52425491, 267.62983254])

In [8]:
# Transforma conjunto de datos con el objeto para imputar
X = imputer.transform(df.drop(['localidad', 'anio', 'cond_resp'], axis=1))
df_tr = pd.DataFrame(X, columns=['actividad_vigorosa_promedio', 'actividad_moderada_promedio', 'actividad_caminar_promedio'])

In [9]:
df_tr = pd.concat([df.drop(['actividad_vigorosa_promedio', 'actividad_moderada_promedio', 'actividad_caminar_promedio'],axis = 1), df_tr], axis=1)
df_tr

,localidad,anio,cond_resp,actividad_vigorosa_promedio,actividad_moderada_promedio,actividad_caminar_promedio
0,10_10_1,2012,2,152.802478,364.524255,267.629833
1,10_10_1,2016,1,152.802478,364.524255,267.629833
2,10_10_29,2012,0,152.802478,364.524255,267.629833
3,10_10_29,2016,1,152.802478,364.524255,267.629833
4,10_10_33,2012,0,152.802478,364.524255,267.629833
...,...,...,...,...,...,...
48773,9_9_74,2016,1,152.802478,364.524255,267.629833
48774,9_9_7777,2012,1,152.802478,364.524255,267.629833
48775,9_9_7777,2016,0,152.802478,364.524255,267.629833
48776,9_9_9999,2012,1,152.802478,364.524255,267.629833


In [10]:
# Divide conjunto de datos con train_test_split
train_set, test_set = train_test_split(df_tr, test_size=0.2, random_state=42)

In [11]:
train_set.shape

(39022, 6)

In [12]:
test_set.shape

(9756, 6)

In [13]:
# Define variables dependientes
Y = df_tr.cond_resp

# Define variables independientes (y = 4 + 3x_1 + ruído gauseano)
X = df_tr[['localidad', 'anio', 'actividad_vigorosa_promedio', 'actividad_moderada_promedio', 'actividad_caminar_promedio']]

In [14]:
# Agregamos x_0 = 1 a cada observación
X_b = np.c_[np.ones((48778, 1)), X]

# Ecuación normal
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
theta_best

In [15]:
# Divide conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)

In [16]:
# Crea objeto de regresión lineal
lin_reg = LinearRegression()

In [17]:
# Ajusta modelo de regresión lineal
lin_reg.fit(X, Y)

# Devuelve intercepto y coeficientes de modelo de regresión lineal
lin_reg.intercept_, lin_reg.coef_

(-111.57937027327277,
 array([-9.93976067e-09,  5.59529334e-02, -2.20225399e-02, -1.14568037e-02,
         3.46136973e-02]))